#Описание задания

В данной проблеме имея некоторую финансовую информацию о клиентах мы должны предсказать customer churn. 

In [ ]:
# импортировать важные библиотеки
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from collections import Counter
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


Для начала загрузим датасеты

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gdsc-datathon/ArbuzTrain.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gdsc-datathon/ArbuzTest.csv")

И увидим что у нас довольно много пустых полей

In [ ]:
train

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27,is_churn
0,301,49300,7,10.0,23.4286,0,9250.0,NaN,NaN,0,...,0.0,0.0,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0
1,9394,58900,7,10.0,5.4286,0,24547.0,NaN,NaN,0,...,0.0,0.0,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0
2,6680,43100,6,10.0,15.3333,0,6017.0,NaN,NaN,0,...,0.0,0.0,0.00,0.00,0.00,0.22,0.00,0.00,0.02,0
3,4774,33900,8,4.0,9.6250,0,7316.0,NaN,NaN,0,...,0.0,0.0,0.01,0.00,0.06,0.35,0.00,0.00,0.17,0
4,2722,37900,9,10.0,14.2222,0,2175.0,10.0,0.0,0,...,0.0,0.0,0.00,0.00,0.04,0.22,0.00,0.00,0.01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10979,5361,22600,7,10.0,7.0000,0,10924.0,10.0,0.0,0,...,0.0,0.0,0.00,0.00,0.00,0.04,0.49,0.00,0.00,0
10980,1172,68600,18,9.4,18.3333,1,28262.0,4.0,1.0,0,...,0.0,0.0,0.00,0.01,0.01,0.34,0.02,0.01,0.02,0
10981,4686,96900,7,10.0,25.1429,0,31532.0,10.0,7.0,17,...,0.0,0.0,0.01,0.01,0.02,0.36,0.10,0.01,0.06,0
10982,2343,11400,18,8.8,1.8333,0,0.0,NaN,NaN,0,...,0.0,0.0,0.00,0.00,0.00,0.24,0.00,0.00,0.00,1


#Подготовить данные к классификации

##Сделать что-то с пустыми полями

In [ ]:
train.head()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27,is_churn
0,301,49300,7,10.0,23.4286,0,9250.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.00,0.00,0.01,0.0,0.00,0
1,9394,58900,7,10.0,5.4286,0,24547.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.03,0.00,0.0,0.00,0
2,6680,43100,6,10.0,15.3333,0,6017.0,NaN,NaN,0,...,0.0,0.0,0.00,0.0,0.00,0.22,0.00,0.0,0.02,0
3,4774,33900,8,4.0,9.6250,0,7316.0,NaN,NaN,0,...,0.0,0.0,0.01,0.0,0.06,0.35,0.00,0.0,0.17,0
4,2722,37900,9,10.0,14.2222,0,2175.0,10.0,0.0,0,...,0.0,0.0,0.00,0.0,0.04,0.22,0.00,0.0,0.01,0


Для начала отделим от данных labels

In [ ]:
train_labels = train['is_churn']
train_data = train.drop('is_churn', axis=1)

И посмотрим каких данных нам не достает

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10984 entries, 0 to 10983
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     10984 non-null  int64  
 1   average_check                 10984 non-null  int64  
 2   number_orders                 10984 non-null  int64  
 3   average_score                 9187 non-null   float64
 4   items_per_order               10984 non-null  float64
 5   is_friend                     10984 non-null  int64  
 6   last_check                    10979 non-null  float64
 7   last_score                    3788 non-null   float64
 8   last_scored_after_order_days  3788 non-null   float64
 9   last_late_by_minutes          10984 non-null  int64  
 10  is_last_order_fullfilled      10984 non-null  int64  
 11  items_share_category_1        10984 non-null  float64
 12  items_share_category_2        10984 non-null  float64
 13  i

Нам нужно каким-то образом заполнить недостающие данные, я буду делать это исходя из некоторых предположений.

- Сумма последнего чека должна быть похожа в среднем на средней чек
- В среднем участники программы лояльности и нет, могут оставлять примерно одинаковый average_score
- Если компания стала плохо/хорошо справляться в целом оценка компании понизится/повысится. Поэтому также имеет смысл заполнить поля last_score средним
- В целом покупатели компании похожи на друг друга по поведению, поэтому они могут оставлять оценку чаще всего через одинаковое число дней, поэтому я заполню все модой поля в last_scored_after_order_days

In [ ]:
train_data['last_check'].fillna(train_data['average_check'].mean(), inplace=True)
train_data['average_score'] = train_data['average_score'].fillna(train_data.groupby('is_friend')['average_score'].transform('mean'))
train_data['last_score'] = train_data['last_score'].fillna(train_data['last_score'].mean())
train_data['last_scored_after_order_days'].fillna(train_data['last_scored_after_order_days'].mode()[0], inplace = True)

Теперь у нас нет пустых полей

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10984 entries, 0 to 10983
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     10984 non-null  int64  
 1   average_check                 10984 non-null  int64  
 2   number_orders                 10984 non-null  int64  
 3   average_score                 10984 non-null  float64
 4   items_per_order               10984 non-null  float64
 5   is_friend                     10984 non-null  int64  
 6   last_check                    10984 non-null  float64
 7   last_score                    10984 non-null  float64
 8   last_scored_after_order_days  10984 non-null  float64
 9   last_late_by_minutes          10984 non-null  int64  
 10  is_last_order_fullfilled      10984 non-null  int64  
 11  items_share_category_1        10984 non-null  float64
 12  items_share_category_2        10984 non-null  float64
 13  i

И теперь данные выглядят так:

In [ ]:
train_data

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_18,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27
0,301,49300,7,10.0,23.4286,0,9250.0,9.477825,0.0,0,...,0.00,0.0,0.0,0.01,0.00,0.00,0.00,0.01,0.00,0.00
1,9394,58900,7,10.0,5.4286,0,24547.0,9.477825,0.0,0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.03,0.00,0.00,0.00
2,6680,43100,6,10.0,15.3333,0,6017.0,9.477825,0.0,0,...,0.07,0.0,0.0,0.00,0.00,0.00,0.22,0.00,0.00,0.02
3,4774,33900,8,4.0,9.6250,0,7316.0,9.477825,0.0,0,...,0.05,0.0,0.0,0.01,0.00,0.06,0.35,0.00,0.00,0.17
4,2722,37900,9,10.0,14.2222,0,2175.0,10.000000,0.0,0,...,0.08,0.0,0.0,0.00,0.00,0.04,0.22,0.00,0.00,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10979,5361,22600,7,10.0,7.0000,0,10924.0,10.000000,0.0,0,...,0.12,0.0,0.0,0.00,0.00,0.00,0.04,0.49,0.00,0.00
10980,1172,68600,18,9.4,18.3333,1,28262.0,4.000000,1.0,0,...,0.11,0.0,0.0,0.00,0.01,0.01,0.34,0.02,0.01,0.02
10981,4686,96900,7,10.0,25.1429,0,31532.0,10.000000,7.0,17,...,0.10,0.0,0.0,0.01,0.01,0.02,0.36,0.10,0.01,0.06
10982,2343,11400,18,8.8,1.8333,0,0.0,9.477825,0.0,0,...,0.58,0.0,0.0,0.00,0.00,0.00,0.24,0.00,0.00,0.00


##Нормализовать данные 

- Имеет смысл отнормализовать предпочтения клиентов в разных категориях продуктов с помощью MinMaxScaling, потому что так мы сможем понять кто наиболее часто покупает в определенной категории среди всех. (Так как число от 0 до 1, нам важно насколько число ближе к 1)
- Та же самая логика работает с average_score и last_score, так как нам важно насколько человек отмечает близко к 10 по шкале от (0-10)
- Остальные числовые колонки имеет смысл нормализовывать с помощью StandardScaler

In [ ]:
preferences = [f"items_share_category_{i}" for i in range(1,28)]
num_columns = ['average_check', 'number_orders', 'items_per_order', 'last_check','last_late_by_minutes', 'last_scored_after_order_days']
scores = ['average_score', 'last_score'] + preferences

In [ ]:
train_data[num_columns].describe()

,average_check,number_orders,items_per_order,last_check,last_late_by_minutes,last_scored_after_order_days
count,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000
mean,60258.749090,16.917243,15.744188,17819.159997,1.867170,1.603696
std,32920.515799,33.871672,7.244092,13674.525587,10.355236,7.270817
min,0.000000,6.000000,1.000000,0.000000,0.000000,0.000000
25%,38000.000000,8.000000,10.800000,8795.750000,0.000000,0.000000
50%,52100.000000,12.000000,14.666700,13865.500000,0.000000,0.000000
75%,73100.000000,20.000000,19.737475,22364.250000,0.000000,0.000000
max,419000.000000,3251.000000,65.857100,202298.000000,478.000000,206.000000


In [ ]:
train_data[scores].describe()

,average_score,last_score,items_share_category_1,items_share_category_2,items_share_category_3,items_share_category_4,items_share_category_5,items_share_category_6,items_share_category_7,items_share_category_8,...,items_share_category_18,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27
count,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,...,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000
mean,9.302935,9.477825,0.013156,0.010122,0.000104,0.048233,0.059496,0.047779,0.020876,0.000136,...,0.072960,0.000066,0.000005,0.000792,0.001331,0.008817,0.233179,0.055713,0.004590,0.034040
std,1.091278,0.763070,0.046579,0.034168,0.001195,0.044195,0.058264,0.095527,0.029875,0.001757,...,0.078148,0.000933,0.000252,0.003546,0.006559,0.015732,0.138920,0.063893,0.016567,0.031593
min,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.300150,9.477825,0.000000,0.000000,0.000000,0.020000,0.020000,0.010000,0.000000,0.000000,...,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140000,0.020000,0.000000,0.010000
50%,9.636400,9.477825,0.000000,0.000000,0.000000,0.040000,0.050000,0.020000,0.010000,0.000000,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220000,0.040000,0.000000,0.030000
75%,10.000000,10.000000,0.000000,0.010000,0.000000,0.060000,0.080000,0.050000,0.030000,0.000000,...,0.090000,0.000000,0.000000,0.000000,0.000000,0.010000,0.320000,0.080000,0.000000,0.050000
max,10.000000,10.000000,0.860000,0.730000,0.040000,0.920000,0.910000,1.000000,1.000000,0.080000,...,1.000000,0.030000,0.020000,0.100000,0.190000,0.470000,1.000000,0.950000,0.750000,0.400000


In [ ]:
scaler = []
min_max_scaler = []

for col in num_columns:
    data = np.array(train_data[col]).reshape(-1,1)
    sc = StandardScaler().fit(data)
    train_data[col] = sc.transform(data)
    scaler.append(sc)

for col in scores:
    data = np.array(train_data[col]).reshape(-1,1)
    minmax = MinMaxScaler().fit(data)
    train_data[col] = minmax.transform(data)
    min_max_scaler.append(minmax)

In [ ]:
train_data.describe()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_18,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27
count,10984.000000,1.098400e+04,1.098400e+04,10984.000000,1.098400e+04,10984.000000,1.098400e+04,10984.000000,1.098400e+04,1.098400e+04,...,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.000000,10984.00000,10984.000000
mean,6864.626639,-8.700655e-17,3.493200e-17,0.912867,3.266789e-17,0.147942,2.529335e-16,0.934728,-1.358467e-17,8.409555e-18,...,0.072960,0.002185,0.000228,0.007921,0.007005,0.018760,0.233179,0.058645,0.00612,0.085099
std,3965.431840,1.000046e+00,1.000046e+00,0.136410,1.000046e+00,0.355059,1.000046e+00,0.095384,1.000046e+00,1.000046e+00,...,0.078148,0.031087,0.012621,0.035461,0.034523,0.033472,0.138920,0.067256,0.02209,0.078982
min,1.000000,-1.830515e+00,-3.223266e-01,0.000000,-2.035433e+00,0.000000,-1.303151e+00,0.000000,-2.205762e-01,-1.803199e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,3443.750000,-6.761669e-01,-2.632775e-01,0.912519,-6.825443e-01,0.000000,-6.599001e-01,0.934728,-2.205762e-01,-1.803199e-01,...,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140000,0.021053,0.00000,0.025000
50%,6856.500000,-2.478430e-01,-1.451793e-01,0.954550,-1.487471e-01,0.000000,-2.891391e-01,0.934728,-2.205762e-01,-1.803199e-01,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220000,0.042105,0.00000,0.075000
75%,10312.250000,3.900861e-01,9.101697e-02,1.000000,5.512725e-01,0.000000,3.323916e-01,1.000000,-2.205762e-01,-1.803199e-01,...,0.090000,0.000000,0.000000,0.000000,0.000000,0.021277,0.320000,0.084211,0.00000,0.125000
max,13729.000000,1.089769e+01,9.548481e+01,1.000000,6.918078e+00,1.000000,1.349131e+01,1.000000,2.811316e+01,4.598200e+01,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


Теперь наши данные готовы для того, чтобы на них обучать классификатор

In [ ]:
final_df_train = train_data.copy()

##Обрабатываем данные для теста

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     2747 non-null   int64  
 1   average_check                 2747 non-null   int64  
 2   number_orders                 2747 non-null   int64  
 3   average_score                 2285 non-null   float64
 4   items_per_order               2747 non-null   float64
 5   is_friend                     2747 non-null   int64  
 6   last_check                    2743 non-null   float64
 7   last_score                    950 non-null    float64
 8   last_scored_after_order_days  950 non-null    float64
 9   last_late_by_minutes          2747 non-null   int64  
 10  is_last_order_fullfilled      2747 non-null   int64  
 11  items_share_category_1        2747 non-null   float64
 12  items_share_category_2        2747 non-null   float64
 13  ite

Повторяем абсолютно те же самые процедуры для того чтобы заполнить пустые поля. Для заполнения используем колонки из train.

In [ ]:
test_data = test.copy()

In [ ]:
test_data['last_check'].fillna(train_data['last_check'].mean())
test_data['average_score'] = test_data['average_score'].fillna(train_data.groupby('is_friend')['average_score'].transform('mean'))
test_data['last_score'] = test_data['last_score'].fillna(train_data['last_score'].mean())
test_data['last_scored_after_order_days'].fillna(train_data['last_scored_after_order_days'].mean(), inplace = True)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     2747 non-null   int64  
 1   average_check                 2747 non-null   int64  
 2   number_orders                 2747 non-null   int64  
 3   average_score                 2747 non-null   float64
 4   items_per_order               2747 non-null   float64
 5   is_friend                     2747 non-null   int64  
 6   last_check                    2743 non-null   float64
 7   last_score                    2747 non-null   float64
 8   last_scored_after_order_days  2747 non-null   float64
 9   last_late_by_minutes          2747 non-null   int64  
 10  is_last_order_fullfilled      2747 non-null   int64  
 11  items_share_category_1        2747 non-null   float64
 12  items_share_category_2        2747 non-null   float64
 13  ite

Нормализуем данные по такому же принципе и приступаем к обучению модели

In [ ]:
for col, sc in zip(num_columns,scaler):
    data = np.array(test_data[col]).reshape(-1,1)
    test_data[col] = sc.transform(data)

for col, mm in zip(scores, min_max_scaler):
    data = np.array(test_data[col]).reshape(-1,1)
    test_data[col] = minmax.transform(data)

In [ ]:
final_df_test = test_data.copy()

In [ ]:
final_df_test.describe()

,client_id,average_check,number_orders,average_score,items_per_order,is_friend,last_check,last_score,last_scored_after_order_days,last_late_by_minutes,...,items_share_category_18,items_share_category_19,items_share_category_20,items_share_category_21,items_share_category_22,items_share_category_23,items_share_category_24,items_share_category_25,items_share_category_26,items_share_category_27
count,2747.000000,2747.000000,2747.000000,2747.000000,2747.000000,2747.000000,2743.000000,2747.000000,2747.000000,2747.000000,...,2747.000000,2747.000000,2747.0,2747.000000,2747.000000,2747.000000,2747.000000,2747.000000,2747.000000,2747.000000
mean,6871.491445,-0.004248,-0.005155,19.618272,-0.022290,0.138697,0.019455,9.690304,0.009996,0.045382,...,0.184365,0.000246,0.0,0.001975,0.003267,0.022288,0.572161,0.138351,0.011185,0.084064
std,3958.698645,1.030046,0.431521,8.321709,1.017031,0.345693,1.708813,10.306026,1.162903,1.188641,...,0.196358,0.003656,0.0,0.009025,0.016403,0.037125,0.343032,0.152067,0.030738,0.075873
min,4.000000,-1.754571,-0.322327,2.281297,-2.035433,0.000000,-1.303151,2.336820,-0.220576,-0.180320,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3393.500000,-0.679205,-0.263278,20.000000,-0.714102,0.000000,-0.660723,2.336820,-0.220576,-0.180320,...,0.075000,0.000000,0.0,0.000000,0.000000,0.000000,0.325000,0.050000,0.000000,0.025000
50%,6922.000000,-0.247843,-0.145179,24.000000,-0.179436,0.000000,-0.294587,2.336820,-0.220576,-0.180320,...,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.550000,0.100000,0.000000,0.075000
75%,10258.000000,0.359709,0.091017,25.000000,0.547086,0.000000,0.326047,25.000000,-0.220576,-0.180320,...,0.225000,0.000000,0.0,0.000000,0.000000,0.025000,0.775000,0.200000,0.000000,0.125000
max,13731.000000,15.663932,8.210265,25.000000,4.729009,1.000000,73.008916,25.000000,21.511121,28.598701,...,2.250000,0.150000,0.0,0.200000,0.425000,0.450000,2.425000,2.150000,0.775000,0.500000


In [ ]:
final_train_data = np.array(final_df_train.copy())
final_train_label = np.array(train_labels.copy())

final_test_data = np.array(final_df_test.copy())

## Делим labeled данные на обучающую и тестовую выборки





Классы сильно не сбалансированные:

In [ ]:
Counter(final_train_label)

Counter({0: 8505, 1: 2479})

При делении на тестовую и трейн используем stratify, чтобы в равно количестве оба класса были задействованы.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_train_data, final_train_label, test_size=0.2, shuffle=True, stratify=final_train_label)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8787, 38), (2197, 38), (8787,), (2197,))

#Строим классификатор

- В данном случае мы работаем с табличными данными
- Наши классы несбалансированные

##Boosting


"XGBoost is a tree based ensemble machine learning algorithm which has higher predicting power and performance and it is achieved by improvisation on Gradient Boosting framework by introducing some accurate approximation algorithms."

In [ ]:
clf=XGBClassifier(max_depth=6, gamma = 0.1, reg_alpha=0.1, min_child_weight = 4, learning_rate=0.2, n_estimators=500, objective='binary:logistic', booster='gbtree')
XGB=clf.fit(X_train,y_train)
prediction=XGB.predict(X_test)
Counter(prediction)

Counter({0: 1882, 1: 315})

In [ ]:
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1701
           1       0.39      0.25      0.30       496

    accuracy                           0.74      2197
   macro avg       0.59      0.57      0.57      2197
weighted avg       0.71      0.74      0.72      2197

[[1508  193]
 [ 374  122]]


In [ ]:
XGB.predict(final_test_data)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
answer = pd.DataFrame()
answer['client_id'] = test['client_id']
answer['is_churn'] = XGB.predict(final_test_data)

In [ ]:
answer

,client_id,is_churn
0,5733,0
1,1559,0
2,8709,0
3,659,0
4,11662,0
...,...,...
2742,948,0
2743,6983,0
2744,8864,0
2745,8877,0


In [ ]:
answer.to_csv('answer.csv', index=False)

##Downsampling

In [ ]:
rus = RandomUnderSampler(sampling_strategy = 0.5)
X_res, y_res = rus.fit_resample(X_train, y_train)
Counter(y_res), Counter(y_train)

(Counter({0: 3966, 1: 1983}), Counter({0: 6804, 1: 1983}))

In [ ]:
clf=XGBClassifier(max_depth = 6, learning_rate=0.3, colsample_bylevel=0.8, n_estimators = 300, objective='binary:logistic', booster='dart')
XGB=clf.fit(X_res, y_res)
prediction=XGB.predict(X_test)
Counter(prediction)
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80      1701
           1       0.34      0.37      0.35       496

    accuracy                           0.69      2197
   macro avg       0.57      0.58      0.58      2197
weighted avg       0.70      0.69      0.70      2197

[[1338  363]
 [ 312  184]]


Not that much improvement. Upsampling with SMOTE works better.



##Upsampling

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train, y_train)

In [ ]:
clf=XGBClassifier(max_depth = 6, learning_rate=0.01, colsample_bytree=0.4, colsample_bylevel=0.7, n_estimators = 500, objective='binary:logistic', booster='gbtree')
XGB=clf.fit(X_res, y_res)
prediction=XGB.predict(X_test)
Counter(prediction)

Counter({0: 1740, 1: 457})

In [ ]:
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1701
           1       0.42      0.38      0.40       496

    accuracy                           0.74      2197
   macro avg       0.62      0.61      0.62      2197
weighted avg       0.73      0.74      0.74      2197

[[1434  267]
 [ 306  190]]


In [ ]:
answer = pd.DataFrame()
answer['client_id'] = test['client_id']
answer['is_churn'] = XGB.predict(final_test_data)
answer.to_csv('answer_nya2.csv', index=False)


At the moment this attempt is best on Kaggle dataset

In [ ]:
XGB.save_model("XGB_best_model.json")